# YOLO TRANSFER LEARNING

---



In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:

!pip install torch torchvision
!pip install pyyaml
!pip install yolov5

import os
import shutil
import random
import yaml
from sklearn.model_selection import train_test_split

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!nvidia-smi


Tue Jun 25 20:43:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import os
import shutil
import random
import yaml
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

# Paths
images_dir = '/content/drive/MyDrive/annotated_images'
annotations_dir = '/content/drive/MyDrive/annotated_images_YOLO_format'
dataset_dir = '/content/dataset'

# Create necessary directories
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'labels/val'), exist_ok=True)

# Get all images and corresponding annotation files
images = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]
annotations = [f.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt') for f in images]

# Split dataset into train and validation sets
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size=0.3, random_state=42)

# Copy files to respective directories
for img, ann in zip(train_images, train_annotations):
    shutil.copy(os.path.join(images_dir, img), os.path.join(dataset_dir, 'images/train', img))
    shutil.copy(os.path.join(annotations_dir, ann), os.path.join(dataset_dir, 'labels/train', ann))

for img, ann in zip(val_images, val_annotations):
    shutil.copy(os.path.join(images_dir, img), os.path.join(dataset_dir, 'images/val', img))
    shutil.copy(os.path.join(annotations_dir, ann), os.path.join(dataset_dir, 'labels/val', ann))

# Create data.yaml file
data_config = {
    'train': os.path.join(dataset_dir, 'images/train'),
    'val': os.path.join(dataset_dir, 'images/val'),
    'nc': 1,  # Number of classes
    'names': ['rat']  # Class names
}

with open(os.path.join(dataset_dir, 'data.yaml'), 'w') as f:
    yaml.dump(data_config, f)

# Install YOLOv5
!pip install yolov5

# Training the YOLOv10 model
from yolov5 import train

train.run(
    data=os.path.join(dataset_dir, 'data.yaml'),  # Path to the dataset configuration file
    imgsz=640,
    batch=16,
    epochs=50,
    weights='yolov5s.pt',  # Pre-trained weights
    project='/content/drive/MyDrive/rat-detection',  # Path to save training results
    name='rat-detector',
    exist_ok=True
)


Mounted at /content/drive


train: weights=yolov5s.pt, cfg=, data=/content/dataset/data.yaml, hyp=../usr/local/lib/python3.10/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/rat-detection, name=rat-detector, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, neptune_resume_id=None, s3_upload_dir=None, upload_dataset=False, hf_model_id=None, hf_token=None, hf_private=False, hf_dataset_id=None, roboflow_token=None, roboflow_upload=False, batch=16
YOLOv5 🚀 2024-6-25 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.

requirements: /usr/local/lib/python3.10/dist-packages/requirements.txt not found, check failed.


100%|██████████| 755k/755k [00:00<00:00, 26.2MB/s]
ClearML: run 'pip install clearml' to automatically track, visualize and remotely train YOLOv5 🚀 in ClearML
Comet: run 'pip install comet_ml' to automatically track and visualize YOLOv5 🚀 runs in Comet
TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/rat-detection', view at http://localhost:6006/
100%|██████████| 14.1M/14.1M [00:00<00:00, 204MB/s]

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4 

Namespace(weights='yolov5s.pt', cfg='', data='/content/dataset/data.yaml', hyp={'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket='', cache=None, image_weights=False, device='', multi_scale=False, single_cls=False, optimizer='SGD', sync_bn=False, workers=8, project='/content/drive/MyDrive/rat-detection', name='rat-detector', exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_ra

In [ ]:
import os
import shutil
import random
import yaml
from sklearn.model_selection import train_test_split

# Paths
images_dir = '/content/drive/MyDrive/annotated_images'
annotations_dir = '/content/drive/MyDrive/annotated_images_YOLO_format'
dataset_dir = '/content/dataset10'

# Create necessary directories
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'labels/val'), exist_ok=True)

# Get all images and corresponding annotation files
images = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]
annotations = [f.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt') for f in images]

# Split dataset into train and validation sets
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size=0.3, random_state=42)

# Copy files to respective directories
for img, ann in zip(train_images, train_annotations):
    shutil.copy(os.path.join(images_dir, img), os.path.join(dataset_dir, 'images/train', img))
    shutil.copy(os.path.join(annotations_dir, ann), os.path.join(dataset_dir, 'labels/train', ann))

for img, ann in zip(val_images, val_annotations):
    shutil.copy(os.path.join(images_dir, img), os.path.join(dataset_dir, 'images/val', img))
    shutil.copy(os.path.join(annotations_dir, ann), os.path.join(dataset_dir, 'labels/val', ann))

# Create data.yaml file
data_config = {
    'train': os.path.join(dataset_dir, 'images/train'),
    'val': os.path.join(dataset_dir, 'images/val'),
    'nc': 1,  # Number of classes
    'names': ['rat']  # Class names
}

with open(os.path.join(dataset_dir, 'data.yaml'), 'w') as f:
    yaml.dump(data_config, f)

# Install YOLOv10 (assuming YOLOv10 is available through pip as yolov10)
!pip install yolov10

